# ALENN - Replication Notebook 
## AR-GARCH Model, Empirical

Donovan Platt
<br>
Mathematical Institute, University of Oxford
<br>
Institute for New Economic Thinking at the Oxford Martin School
<br>
<br>
Copyright (c) 2020, University of Oxford. All rights reserved.
<br>
Distributed under a BSD 3-Clause licence. See the accompanying LICENCE file for further details.

# 1. Modules and Packages
Load all required modules and packages.

In [1]:
# Import the ALENN ABM Estimation Package
import alenn

# Import Numerical Computation Libraries
import numpy as np
import pandas as pd

# Import General Mathematical Libraries
from scipy import stats

# Import System Libraries
import os
import logging

# Import Data Storage Libraries
import pickle as pkl

Using TensorFlow backend.


In [2]:
# Disable Tensorflow Deprecation Warnings
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Tensorflow 2.x deprecates many Tensorflow 1.x methods, causing Tensorflow 1.15.0 to output a large number 
# of deprecation warnings when performing the first likelihood calculation. This can be very distracting,
# leading us to disable them.

# 2. Estimation Experiments
Replication of the MDN experiments. Note that here we generate only a single Markov Chain as opposed to the 5 considered in the original paper.

## 2.1. Pseudo-Empirical Data

### Model Specification

In [4]:
# Specify the Simulated Data Characteristics
T_emp = 2000    # Pseudo-empirical series length 
T_sim = 2000    # Length of each Monte Carlo replication
n = 50          # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = alenn.models.ar_garch(0, 0.2, 0.25, 0.1, 0.5, 0.2, T_emp, 1, 1)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return alenn.models.ar_garch(0, theta[0], theta[1], theta[2], theta[3], theta[4], T_sim, n, 7)

# Define Prior
priors = [stats.uniform(loc = -1.5, scale = 3).pdf,
          stats.uniform(loc = -1.5, scale = 3).pdf,
          stats.uniform(loc = 0, scale = 2).pdf,
          stats.uniform(loc = 0, scale = 2).pdf,
          stats.uniform(loc = 0, scale = 2).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-1.5, -1.5, 0, 0, 0])
theta_upper = np.array([1.5, 1.5, 2, 2, 2])

### Posterior Specification

In [5]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 5 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 2000 observations in total.
-------------------------------------------------

### Sampler Specification

In [8]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 15000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         15000               
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                 -1.5          1.5
2                 -1.5          1.5
3                  0.0          2.0
4                  0.0          2.0
5                  0.0          2.0
----------------------------------------------------------------------------



### Result Processing

In [7]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 10000)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.203240             0.022455
2                0.271714             0.018937
3                0.074780             0.008301
4                0.624457             0.049814
5                0.219923             0.042379


## 2.2. FTSE 100

### Model Specification

In [3]:
# Specify the Simulated Data Characteristics
T_sim = 2000    # Length of each Monte Carlo replication
n = 50          # Number of Monte Carlo replications

# Load the Empirical Data
with open('data/FTSE_100_Log_Returns', 'rb') as f:
    empirical = pkl.load(f)

# The above empirical data corresponds to log-returns derived from the closing prices of the 
# FTSE 100 from 03-01-2012 to 30-12-2019. The data is widely and freely available from 
# various sources.

# Determine the Empirical Variance
T_emp = len(empirical)
sigma_emp = empirical.var()

# Define the Candidate Model Function
def model(theta):
    return alenn.models.ar_garch(0, theta[0], theta[1], theta[2], theta[3], theta[4], T_sim, n, 7)

# Define Prior
priors = [stats.uniform(loc = -1.5, scale = 3).pdf,
          stats.uniform(loc = -1.5, scale = 3).pdf,
          stats.uniform(loc = 0, scale = 2 * sigma_emp).pdf,
          stats.uniform(loc = 0, scale = 2).pdf,
          stats.uniform(loc = 0, scale = 2).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-1.5, -1.5, 0, 0, 0])
theta_upper = np.array([1.5, 1.5, 2 * sigma_emp, 2, 2])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 5 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 2019 observations in total.
-------------------------------------------------

### Sampler Specification

In [7]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 15000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         15000               
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                 -1.5     1.500000
2                 -1.5     1.500000
3                  0.0     0.000141
4                  0.0     2.000000
5                  0.0     2.000000
----------------------------------------------------------------------------



### Result Processing

In [6]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 10000)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1            3.765961e-04         2.529855e-02
2           -1.248016e-02         2.691702e-02
3            9.727995e-07         4.765273e-07
4            9.626374e-02         2.294433e-02
5            8.932834e-01         2.816114e-02


## 2.2. Nikkei 225

### Model Specification

In [3]:
# Specify the Simulated Data Characteristics
T_sim = 2000    # Length of each Monte Carlo replication
n = 50          # Number of Monte Carlo replications

# Load the Empirical Data
with open('data/Nikkei_225_Log_Returns', 'rb') as f:
    empirical = pkl.load(f)

# The above empirical data corresponds to log-returns derived from the closing prices of the 
# Nikkei 225 from 01-03-2011 to 30-12-2019. The data is widely and freely available from 
# various sources.

# Determine the Empirical Variance
T_emp = len(empirical)
sigma_emp = empirical.var()

# Define the Candidate Model Function
def model(theta):
    return alenn.models.ar_garch(0, theta[0], theta[1], theta[2], theta[3], theta[4], T_sim, n, 7)

# Define Prior
priors = [stats.uniform(loc = -1.5, scale = 3).pdf,
          stats.uniform(loc = -1.5, scale = 3).pdf,
          stats.uniform(loc = 0, scale = 2 * sigma_emp).pdf,
          stats.uniform(loc = 0, scale = 2).pdf,
          stats.uniform(loc = 0, scale = 2).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-1.5, -1.5, 0, 0, 0])
theta_upper = np.array([1.5, 1.5, 2 * sigma_emp, 2, 2])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 5 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 2165 observations in total.
-------------------------------------------------

### Sampler Specification

In [7]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 15000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         15000               
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                 -1.5     1.500000
2                 -1.5     1.500000
3                  0.0     0.000345
4                  0.0     2.000000
5                  0.0     2.000000
----------------------------------------------------------------------------



### Result Processing

In [6]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 10000)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1               -0.010300         2.928748e-02
2                0.045938         2.950828e-02
3                0.000001         5.099562e-07
4                0.101682         1.649215e-02
5                0.894102         1.825518e-02
